# _*Repetition Codes*_

***

### Contributors
James R. Wootton, IBM Research

***

In quantum error correction we take many noisy qubits (which we call *physical qubits*) and use them to store a smaller number of *logical qubits*. The error correction procedure is designed to detect and correct for the effects of noise. This makes the logical qubits much less noisy and more reliable than the physical ones from which they are built.

The repetition code is a simple example of quantum error correction, in which a logical bit is stored rather than a logical qubit. A given instance of the repetition code is defined by two numbers, which we call $d$ and $T$.

The $d$ parameter determines how many physical qubits are used. The actual information about the logical bit is stored in $d$ qubits (which we'll call _code qubits_). The encoding is done in a very simple way: to encode a `0`, all these qubits are set in the state $|0\rangle$, to encode a `1` they are set to $|1\rangle$. To readout the value, you can just look at any qubit. To read it out in a way that protects agains single qubit errors, you can read out all the qubits and take a majority vote.

It is not only at readout that we can extract useful information that will help us correct errors. We can also extract information while the logical bit sits idle, or even while it is involved in computation.

Since this is an exercise in quantum error correction, we will do this in a way that would also work for logical *qubits*. Specifically, our method of extracting information about errors must not extract any information regarding the stored logical information. In the quantum case, this is required so that we do not disturb superposition states of the stored qubits.

We will extract information over $T$ rounds of _syndrome measurements_. For the repetition code, these are based on the fact that all code qubits should be in the same state (all $|0\rangle$ or all $|1\rangle$). Any departure from this is therefore a signature of error. Specifically, we imagine our $d$ code qubits sitting along a line. We will then perform a measurement on each pair of neighbouring code qubits. This will tell us whether they are the same or different, without extracting any information on what their values are.

The implementation of these measurements requires $d-1$ additional qubits, which we will call _link qubits_ for the repetition code. By peforming two cnots between the pair of code qubits and a corresponding link qubit, the required information (and only the required information) is placed on the link qubit and can then be measured.

In [1]:
from qiskit import *
cq = QuantumRegister(2,'code_qubit')
lq = QuantumRegister(1,'link_qubit')
qc = QuantumCircuit(cq,lq)
qc.cx(cq[0],lq[0])
qc.cx(cq[1],lq[0])
print(qc)

                           
code_qubit_0: |0>──■───────
                   │       
code_qubit_1: |0>──┼────■──
                 ┌─┴─┐┌─┴─┐
link_qubit_0: |0>┤ X ├┤ X ├
                 └───┘└───┘


Qiskit Ignis provides tools for creating and testing repetition codes. The first thing we need to do is import them.

In [2]:
import sys
sys.path.append('../')
from TopologicalQEC import repetition_code, decoder, execute_codes

## Creating a repetition code

The `repetition_code` class creates a code for given values of $d$ and $T$

In [3]:
d = 3
T = 2
code = repetition_code(d,T)

With this we can inspect various properties of the code, such as the names of the qubit registers used for the code and ancilla qubits.

In [4]:
code.qubit_registers

{'code_qubit', 'link_qubit'}

These registers are also attributes of the `repetition_code` object.

In [5]:
code.code_qubit

QuantumRegister(3, 'code_qubit')

You can also access the quantum circuits that implement the code. Two of these are produced: one for each of the two possible logical bit values.

In [6]:
for log in [0,1]:
    print('\n========= logical',log,'=========\n')
    print( code.circuit[log] )


========= logical 0 =========

                                        ┌───┐┌───┐┌─┐      ░                   »
     link_qubit_0: |0>──────────────────┤ X ├┤ X ├┤M├─|0>──░───────────────────»
                      ┌───┐┌───┐┌─┐     └─┬─┘└─┬─┘└╥┘      ░ ┌───┐┌───┐┌─┐     »
     link_qubit_1: |0>┤ X ├┤ X ├┤M├─|0>───┼────┼───╫───────░─┤ X ├┤ X ├┤M├─|0>─»
                      └─┬─┘└─┬─┘└╥┘       │    │   ║       ░ └─┬─┘└─┬─┘└╥┘     »
     code_qubit_0: |0>──┼────┼───╫────────■────┼───╫───────░───┼────┼───╫──────»
                        │    │   ║             │   ║       ░   │    │   ║      »
     code_qubit_1: |0>──■────┼───╫─────────────■───╫───────░───■────┼───╫──────»
                             │   ║                 ║       ░        │   ║      »
     code_qubit_2: |0>───────■───╫─────────────────╫───────░────────■───╫──────»
                                 ║                 ║       ░            ║      »
round_0_link_bit_0: 0 ═══════════╬═════════════════╩════════════════════╬════

## Building a custom repetition code

You can also implement measurement rounds and logical `x` gates yourself. For example, let's set up a code with no syndrome measurement rounds.

In [7]:
empty_code = repetition_code(3,0)

This does nothing but set up two circuits for the two logical encoded states. There are no syndrome measurement rounds, and no final readout.

In [8]:
def print_circuits(code):
    for log in [0,1]:
        print('\n========= logical',log,'=========\n')
        print( code.circuit[log] )
            
print_circuits(empty_code)


========= logical 0 =========

                 
link_qubit_0: |0>
                 
link_qubit_1: |0>
                 
code_qubit_0: |0>
                 
code_qubit_1: |0>
                 
code_qubit_2: |0>
                 

========= logical 1 =========

                                 ░ 
link_qubit_0: |0>────────────────░─
                                 ░ 
link_qubit_1: |0>────────────────░─
                           ┌───┐ ░ 
code_qubit_0: |0>──────────┤ X ├─░─
                      ┌───┐└───┘ ░ 
code_qubit_1: |0>─────┤ X ├──────░─
                 ┌───┐└───┘      ░ 
code_qubit_2: |0>┤ X ├───────────░─
                 └───┘           ░ 


We can add a round using the `syndrome_measurement()` method.

In [9]:
empty_code.syndrome_measurement()
print_circuits(empty_code)


========= logical 0 =========

                                        ┌───┐┌───┐┌─┐      ░ 
     link_qubit_0: |0>──────────────────┤ X ├┤ X ├┤M├─|0>──░─
                      ┌───┐┌───┐┌─┐     └─┬─┘└─┬─┘└╥┘      ░ 
     link_qubit_1: |0>┤ X ├┤ X ├┤M├─|0>───┼────┼───╫───────░─
                      └─┬─┘└─┬─┘└╥┘       │    │   ║       ░ 
     code_qubit_0: |0>──┼────┼───╫────────■────┼───╫───────░─
                        │    │   ║             │   ║       ░ 
     code_qubit_1: |0>──■────┼───╫─────────────■───╫───────░─
                             │   ║                 ║       ░ 
     code_qubit_2: |0>───────■───╫─────────────────╫───────░─
                                 ║                 ║       ░ 
round_0_link_bit_0: 0 ═══════════╬═════════════════╩═════════
                                 ║                           
round_0_link_bit_1: 0 ═══════════╩═══════════════════════════
                                                             

========= logical 1 =========

      

A logical `x` operation can be added using the `x()` method. 

In [10]:
empty_code.x()
print_circuits(empty_code)


========= logical 0 =========

                                        ┌───┐┌───┐┌─┐      ░                 ░ 
     link_qubit_0: |0>──────────────────┤ X ├┤ X ├┤M├─|0>──░─────────────────░─
                      ┌───┐┌───┐┌─┐     └─┬─┘└─┬─┘└╥┘      ░                 ░ 
     link_qubit_1: |0>┤ X ├┤ X ├┤M├─|0>───┼────┼───╫───────░─────────────────░─
                      └─┬─┘└─┬─┘└╥┘       │    │   ║       ░           ┌───┐ ░ 
     code_qubit_0: |0>──┼────┼───╫────────■────┼───╫───────░───────────┤ X ├─░─
                        │    │   ║             │   ║       ░      ┌───┐└───┘ ░ 
     code_qubit_1: |0>──■────┼───╫─────────────■───╫───────░──────┤ X ├──────░─
                             │   ║                 ║       ░ ┌───┐└───┘      ░ 
     code_qubit_2: |0>───────■───╫─────────────────╫───────░─┤ X ├───────────░─
                                 ║                 ║       ░ └───┘           ░ 
round_0_link_bit_0: 0 ═══════════╬═════════════════╩═══════════════════════════
        

This also has a `log` kwarg which can be set to `0` or `1`, which applies the `x` only to the circuit for that initialized value.

Final readout is then done with the `readout()` method.

In [11]:
empty_code.readout()
print_circuits(empty_code)


========= logical 0 =========

                                        ┌───┐┌───┐┌─┐      ░                 ░ »
     link_qubit_0: |0>──────────────────┤ X ├┤ X ├┤M├─|0>──░─────────────────░─»
                      ┌───┐┌───┐┌─┐     └─┬─┘└─┬─┘└╥┘      ░                 ░ »
     link_qubit_1: |0>┤ X ├┤ X ├┤M├─|0>───┼────┼───╫───────░─────────────────░─»
                      └─┬─┘└─┬─┘└╥┘       │    │   ║       ░           ┌───┐ ░ »
     code_qubit_0: |0>──┼────┼───╫────────■────┼───╫───────░───────────┤ X ├─░─»
                        │    │   ║             │   ║       ░      ┌───┐└───┘ ░ »
     code_qubit_1: |0>──■────┼───╫─────────────■───╫───────░──────┤ X ├──────░─»
                             │   ║                 ║       ░ ┌───┐└───┘      ░ »
     code_qubit_2: |0>───────■───╫─────────────────╫───────░─┤ X ├───────────░─»
                                 ║                 ║       ░ └───┘           ░ »
round_0_link_bit_0: 0 ═══════════╬═════════════════╩═════════════════════════

## Running a repetition code

To run the code we use the `get_syndrome()` method. This runs the circuit for one of the logical values, as specified by the `log` kwarg. This has `log=0` by default.


The `get_syndrome()` method also has kwargs that are passed on to Qiskit's `execute()` function, such as `shots=1024` and `backend=Aer.get_backend('qasm_simulator')`. 

In [12]:
code = repetition_code(5,2)

for log in [0,1]:
    print('\nresult for logical',log,'\n')
    print( code.get_syndrome(log=log,shots=1) )


result for logical 0 

{'0 0  0000 0000 0000': 1}

result for logical 1 

{'1 1  0000 0000 0000': 1}


The result is a dictionary whose keys are bit strings that represent outputs of the circuit, and the values represent the number of shots for which this output occurred.

The strings are not the direct output from the circuits. They have been processed to take the form that helps us correct errors. Here's a short guided tour.

* The `0 0` to the far left for the logical `0` result, and the `1 1` to the far left of the logical `1`, are the logical readout. Any code qubit could be used for this readout, since they should (without errors) all be equal. So we could have just one result here, for one arbitarily chosen code qubit. Or we could have $d$, one for each qubit. Instead we have two, from the two qubits at either end of the line. This is because it works best with the decoder (which we'll use later). In the absence of errors, these two values will always be equal.

* The following `0000` is the $d-1$ results of the syndrome measurements for the first round.A `0` implies that the corresponding pair of qubits where the same, and `1` implies different. There are $d-1$ results because the line of $d$ code qubits has $d-1$ possible nighbouring pairs. In the absence of errors, they will all be `0`.

* The `0000` that follows that is the syndrome change between the first and second rounds. It is therefore the bitwise `OR` of the syndrome measurement results from the second round with those from the first. In the absence of errors, they will all be `0`.

* Subsequent blocks follow the same formula, though the last requires some comment. This is not measured using the standard method (with a link qubit). Instead it is calculated from the final readout measurement of all code qubits. Again it is presented as a syndrome change, and will be all `0` in the absence of errors. This is the $T+1$th block of syndrome measurements since, as it is not done in the same way as the others, it is not counted among the $T$ syndrome measurement rounds.

**Example 1:** `0 0  0110 0000 0000` would represent a $d=5$, $T=2$ repetition code with encoded `0`. The syndrome shows that (most likely) the middle code qubit was flipped by an error before the first measurement round. This causes it to disagree with both neighbouring code qubits for the rest of the circuit. This is shown by the syndrome in the first round, but the blocks for subsequent rounds do not report it as it no longer represents a change. Other sets of errors could also have caused this syndrome, but they would need to be more complex and so presumably less likely.

**Example 2:** `0 0  0010 0010 0000` would represent a $d=5$, $T=2$ repetition code with encoded `0`. Here one of the syndrome measurements reported a difference between two code qubits in the first round, leading to a `1`. The next round did not see the same effect, and so resulted in a `0`. However, since this disagreed with the previous result for the same syndrome measurement, and since we track syndrome changes, this change results in another `1`. Subsequent rounds also do not detect anything, but this no longer represents a change and hence results in a `0` in the same position. Most likely the measurement result leading to the first `1` was an error.

**Example 3:** `0 1  0000 0001 0000` would represent a $d=5$, $T=2$ repetition code with encoded `1`. A code qubit on the end of the line is flipped before the second round of syndrome measurements. This is detected by only a single syndrome measurement, because it is on the end of the line. For the same reason, it also disturbs one of the logical readouts.

Note that in all these examples, a single error causes exactly two characters in the string to change from the value it would have with no errors. This is, in fact, the reason why the logical output consists of both endpoints. It is a property that will be used by the decoder.

To see the effects of noise, we need to specify a noise model. For example, we let's set up a simple noise model with gate and measurement errors.

In [13]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

def get_noise(p_meas=0.08,p_gate = 0.04):

    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = depolarizing_error(p_gate, 1)
    error_gate2 = error_gate1.kron(error_gate1)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure")
    noise_model.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"])
        
    return noise_model

In [14]:
noise_model = get_noise()

This can then be passed to `get_syndrome` to generate noisy results.

In [15]:
code = repetition_code(5,2)

for log in [0,1]:
    print('\nresult for logical',log,'\n')
    print( code.get_syndrome(log=log,noise_model=noise_model,shots=100) )


result for logical 0 

{'0 0  0000 0000 0110': 2, '0 0  0011 0010 0001': 1, '1 0  0010 0011 0101': 1, '0 0  0000 0010 0100': 2, '0 1  0000 0001 1100': 1, '0 0  0000 0010 0010': 2, '0 1  0000 0000 1101': 3, '0 0  0000 1000 1000': 2, '0 1  1001 1010 0001': 1, '0 0  0100 0100 0110': 1, '0 0  0000 0100 0100': 1, '0 0  0100 0000 0100': 1, '0 0  0010 0100 0000': 1, '0 0  0000 0010 0001': 2, '0 0  0110 0110 0101': 1, '0 0  1010 1000 0100': 1, '0 0  0000 0000 1100': 4, '0 0  0100 1100 1000': 2, '0 0  0000 0000 0000': 14, '0 0  0001 0010 0000': 1, '0 0  0000 1000 1011': 1, '0 0  0011 1101 1011': 1, '0 0  0010 0011 0010': 1, '0 0  0000 0000 1111': 1, '0 0  0010 0010 0000': 4, '1 0  0000 1000 0000': 1, '0 0  0000 0010 0111': 1, '1 0  0000 0001 1001': 1, '1 0  0001 0010 1000': 1, '0 0  0001 0011 0010': 1, '0 0  0000 0001 0001': 2, '1 0  0000 0000 1000': 1, '0 1  0000 0010 0000': 1, '0 0  0010 0010 0110': 1, '0 1  0000 0000 0001': 3, '0 1  1000 0111 0001': 1, '0 0  0010 0110 1011': 1, '0 1  1000 1

Here the non-noisy results from before are the most likely, since these outputs are found to occur in around a third of the total samples. The other half of the samples are distributed among other possibilities with non-trivial syndromes.

The `get_syndrome()` method gets a result for just one logical value of just one code. However, we will sometimes want to batch many jobs together. This is done with the `execute_codes()` function. This takes a list of codes, and returns their syndrome in the form of a dictionary. The keys for this are in the from of a tuple, which contain the corresponding code object and logical value for the result.

Like `get_syndrome()`, this has kwargs that it passes on to the `execute()` function of Qiskit Terra.

In [16]:
codes = []
for d in range(3,6):
    codes.append( repetition_code(d,2) )
    
all_results = execute_codes(codes)

for code in codes:
    print('\n\ncode with d =',code.d,',T =',code.T)
    for log in [0,1]:
        print('\nresult for logical',log,'\n')
        print( all_results[(code,log)] )



code with d = 3 ,T = 2

result for logical 0 

{'0 0  00 00 00': 1024}

result for logical 1 

{'1 1  00 00 00': 1024}


code with d = 4 ,T = 2

result for logical 0 

{'0 0  000 000 000': 1024}

result for logical 1 

{'1 1  000 000 000': 1024}


code with d = 5 ,T = 2

result for logical 0 

{'0 0  0000 0000 0000': 1024}

result for logical 1 

{'1 1  0000 0000 0000': 1024}


Now let's run on a real device.

In [17]:
from qiskit import IBMQ
IBMQ.load_accounts()

d = 5
code = repetition_code(d,1)
backend = IBMQ.get_backend('ibmq_16_melbourne')
code.get_syndrome(backend=backend,shots=1024)

../TopologicalQEC.py:54: Warning: The transpiler has changed the number of two qubit gates (27 instead of 8 for a distance 5 code).
  warnings.warn("The transpiler has changed the number of two qubit gates (" +str(cx_num)+ " instead of " +str(2*(self.d-1))+ " for a distance " +str(self.d)+ " code).", Warning)


The compiler doesn't always find a way to run the circuit directly, and will instead use techniques such as swap gates to execute it within the constraints of the device. This is the reason for the warning that appears above: it tells us that additional two qubit gates were added by the compilation.

When studying error correction on real devices, we do not want this to happen. We typically want to specify exactly which qubits on the device correspond to which qubits in the code. This is done using an `initial_layout` kwarg for `get_syndrome()`, and something similar for `execute_codes()`.

For this we need to find a suitable path of qubits across the device.

In [18]:
path = [0,1,2,3,4,5,6,8,9,10,11,12,13]

Then remind ourselves what the registers are called.

In [19]:
code.qubit_registers

{'code_qubit', 'link_qubit'}

Then we can construct the required `initial_layout` to avoid the error message.

In [20]:
initial_layout = {}
for j in range(d):
    initial_layout[('code_qubit',j)] = ('q',2*j)
for j in range(d-1):
    initial_layout[('link_qubit',j)] = ('q',2*j+1)

## Decoding a repetition code

Noisy results can change the logical value at readout, and so affect our ability to read out the logical qubit. This can be mitigated by looking at the syndrome values. These can tell us whether or not the logical values are most likely to have changed, and so allow us to correct for the errors. To process of analysing the syndrome to correct for the errors is called 'decoding'. We do this by building a decoding object for our code.

In [21]:
dec = decoder( repetition_code(5,2) )

This analyses the code by seeing how different kinds of error change the output. With this information, along with a decoding algorithm, we can determine what the logical value is most likely to have been.

For example, let's use the 'matching' algorithm to decode, which is based on minimum weight perfect matching. This takes specific output strings as input. We'll give it the simple example string `'1 0  0001 1000 1000'`, for a logical `1` that has suffered two errors.

In [22]:
dec.matching('1 0  0001 1000 1000')

'1 1'

The output is what the logical part should have been. As you can see, the decoder correctly determined that the readout should have been of a logical `1`.

When we take many samples, we can determine the probability with which the decoder is incorrect. This should decrease exponentially as the code size is increased, since the configurations of noise that fool the decoder become less likely. The probability of a logical error is calculated using the `logical_prob()` method. This runs matching by default, but other algorithms can be specified by the `algorithm` kwarg. Below we also use 

In [23]:
for d in range(3,8):
    
    code = repetition_code(d,2)
    
    dec = decoder(code)

    for log in [0,1]:

        results = code.get_syndrome(noise_model=noise_model,log=log,shots=8192)

        print('d =',d,',log =',log,',logical error prob (matching) =', dec.logical_prob(results,log))
        print('d =',d,',log =',log,',logical error prob (postselection) =', dec.logical_prob(results,log,algorithm='postselect'))
        print('')
    print('')

d = 3 ,log = 0 ,logical error prob (matching) = 0.06201171875
d = 3 ,log = 0 ,logical error prob (postselection) = 0.0011675423234092236

d = 3 ,log = 1 ,logical error prob (matching) = 0.0584716796875
d = 3 ,log = 1 ,logical error prob (postselection) = 0.001782001782001782


d = 4 ,log = 0 ,logical error prob (matching) = 0.0443115234375
d = 4 ,log = 0 ,logical error prob (postselection) = 0.0

d = 4 ,log = 1 ,logical error prob (matching) = 0.0416259765625
d = 4 ,log = 1 ,logical error prob (postselection) = 0.0


d = 5 ,log = 0 ,logical error prob (matching) = 0.02685546875
d = 5 ,log = 0 ,logical error prob (postselection) = 0.0

d = 5 ,log = 1 ,logical error prob (matching) = 0.02587890625
d = 5 ,log = 1 ,logical error prob (postselection) = 0.0


d = 6 ,log = 0 ,logical error prob (matching) = 0.02001953125
d = 6 ,log = 0 ,logical error prob (postselection) = 0.0

d = 6 ,log = 1 ,logical error prob (matching) = 0.018310546875
d = 6 ,log = 1 ,logical error prob (postselection) = 